In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import yaml
from utils import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
config_filename = "D:/Projects/SANDAG/client_sandag_off_model_calculators/transit_signal_priority/data/config.yml"

In [4]:
if not os.path.exists(config_filename):
    msg = "Configuration file doesn't exist at: {}".format(config_filename)
    raise ValueError(msg)

with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
# inputs
highway_load_am_file = config['inputs']['highway_load_am_file']
highway_load_pm_file = config['inputs']['highway_load_pm_file']
highway_network_shapefile = config['inputs']['highway_network_shapefile']

transit_flow_file = config['inputs']['transit_flow_file']
transit_aggflow_file = config['inputs']['transit_aggflow_file']
transit_routes_file = config['inputs']['transit_routes_file']
transit_stops_file = config['inputs']['transit_stops_file']
transit_links_file = config['inputs']['transit_links_file']
transit_onoff_file = config['inputs']['transit_onoff_file']

tsp_strategy_file = config['inputs']['tsp_strategy_file']
emission_factors_file = config['inputs']['emission_factors_file']

In [6]:
# outputs
output_dir = config['outputs']['output_dir']
output_results_filename = config['outputs']['output_file_name']

In [7]:
# parameters
scen_year = config['parameters']['scen_year']
periods_length = config['parameters']['periods_length_in_hr']
AVG_DWELL_TIME_MIN = config['parameters']['avg_dwell_time_mins']
GREEN_TO_CYCLE_WITHOUT_TSP = config['parameters']['green_to_cycle_without_tsp']
GREEN_TO_CYCLE_PRIMARY_WITH_TSP = config['parameters']['green_to_cycle_primary_with_tsp']
GREEN_TO_CYCLE_SECONDARY_WITH_TSP = config['parameters']['green_to_cycle_secondary_with_tsp']
AVG_CYCLE_LENGTH = config['parameters']['avg_cycle_length_seconds']
TT_ELASTICITY_TRANSIT_RIDERSHIP = config['parameters']['elasticity_transit_ridership']
AVG_AUTO_OCCUPANCY = config['parameters']['avg_auto_occupancy']

In [8]:
#temp_df["am_total_hourly_transit_dir1"], temp_df["am_total_hourly_transit_dir2"] = zip(*temp_df['hwycov_id'].apply(lambda x: get_transit_vehicles_on_link(x, transit_routes_df, transit_links_df, "AM")))

In [9]:
with open(tsp_strategy_file, "r") as yml_file:
    tsp_config = yaml.safe_load(yml_file)
    tsp_strategy_dict = tsp_config['intersections']
    
number_of_tsp_intersections = len(tsp_strategy_dict["intersection_node_id"])

In [10]:
tsp_strategy_dict

{'intersection_node_id': [15381, 15385, 15387],
 'mainline_corridor_name': ['EL CAJON', 'EL CAJON', 'EL CAJON']}

In [11]:
# read inputs 
links_df = gpd.read_file(highway_network_shapefile)

highway_load_am_df = pd.read_csv(highway_load_am_file)
highway_load_pm_df = pd.read_csv(highway_load_pm_file)

transit_flow_df = pd.read_csv(transit_flow_file)
transit_aggflow_df = pd.read_csv(transit_aggflow_file)
transit_routes_df = pd.read_csv(transit_routes_file)
transit_stops_df = pd.read_csv(transit_stops_file)
transit_links_df = pd.read_csv(transit_links_file)
transit_onoff_df = pd.read_csv(transit_onoff_file)

emission_df = pd.read_excel(emission_factors_file)

In [12]:
highway_load_data_dict = {"AM": highway_load_am_df, "PM": highway_load_pm_df}

In [13]:
corridor_links_df = get_corridor_links(links_df, tsp_strategy_dict)

In [14]:
corridor_links_df

,hwycov_id,link_name,from_node,to_node,corridor,intersection,approach
0,6557,EL CAJON,15380,15381,Primary,1.0,AB
1,6558,EL CAJON,15381,15382,Primary,1.0,BA
2,14726,TEXAS,15278,15381,Secondary,1.0,AB
3,9982,TEXAS,15381,15483,Secondary,1.0,BA
4,6559,EL CAJON,30914,15385,Primary,2.0,AB
5,3659,EL CAJON,15385,15386,Primary,2.0,BA
6,14716,UTAH,15280,15385,Secondary,2.0,AB
7,14735,UTAH,15385,15488,Secondary,2.0,BA
8,3660,EL CAJON,15386,15387,Primary,3.0,AB
9,5236,EL CAJON,15387,15389,Primary,3.0,BA


In [15]:
query = "corridor == 'Primary' and intersection > 0"
primary_links_df = corridor_links_df.query(query, engine="python")

In [16]:
primary_links_df

,hwycov_id,link_name,from_node,to_node,corridor,intersection,approach
0,6557,EL CAJON,15380,15381,Primary,1.0,AB
1,6558,EL CAJON,15381,15382,Primary,1.0,BA
4,6559,EL CAJON,30914,15385,Primary,2.0,AB
5,3659,EL CAJON,15385,15386,Primary,2.0,BA
8,3660,EL CAJON,15386,15387,Primary,3.0,AB
9,5236,EL CAJON,15387,15389,Primary,3.0,BA


In [17]:
# calculate average transit headway for the intersection
am_average_transit_headway = get_average_transit_headway(primary_links_df, transit_routes_df, transit_links_df, AVG_CYCLE_LENGTH, "AM")
pm_average_transit_headway = get_average_transit_headway(primary_links_df, transit_routes_df, transit_links_df, AVG_CYCLE_LENGTH, "PM")

In [18]:
am_average_transit_headway, pm_average_transit_headway

(2.38, 2.38)

In [19]:
# calculate average trip length of transit riders in the corridor
am_avg_transit_trip_length = get_average_transit_trip_length(primary_links_df, transit_links_df, transit_flow_df, transit_onoff_df, "AM")
pm_avg_transit_trip_length = get_average_transit_trip_length(primary_links_df, transit_links_df, transit_flow_df, transit_onoff_df, "PM")

In [20]:
am_avg_transit_trip_length, pm_avg_transit_trip_length

(2.88, 2.63)

In [21]:
# calculate average transit flow at intersection
am_avg_transit_flow = get_average_transit_flow(primary_links_df, transit_aggflow_df, "AM")
pm_avg_transit_flow = get_average_transit_flow(primary_links_df, transit_aggflow_df, "PM")

In [22]:
am_avg_transit_flow, pm_avg_transit_flow

(1218.91, 1941.25)

In [23]:
# calculate total stops along the primary corridor
ab_transit_stops, ba_transit_stops = get_corridor_transit_stops(corridor_links_df, transit_routes_df, transit_stops_df)

In [24]:
ab_transit_stops, ba_transit_stops

(3.3333333333333335, 3.0)

In [25]:
# get link attributes for all links (primary and secondary) in the corridor
corridor_links_df = get_link_attributes(corridor_links_df, highway_load_data_dict, transit_aggflow_df)

In [26]:
corridor_links_df

,hwycov_id,link_name,from_node,to_node,corridor,intersection,approach,ab_am_time,ba_am_time,ab_am_flow,ba_am_flow,ab_am_auto_flow,ba_am_auto_flow,ab_am_truck_flow,ba_am_truck_flow,ab_am_capacity,ba_am_capacity,ab_am_transit_flow,ba_am_transit_flow,ab_pm_time,ba_pm_time,ab_pm_flow,ba_pm_flow,ab_pm_auto_flow,ba_pm_auto_flow,ab_pm_truck_flow,ba_pm_truck_flow,ab_pm_capacity,ba_pm_capacity,ab_pm_transit_flow,ba_pm_transit_flow
0,6557,EL CAJON,15380,15381,Primary,1.0,AB,0.764786,0.145249,1979.406860,2690.896973,1958.778651,2650.571520,20.628209,40.325453,13307.742048,13707.735274,398.961856,783.377114,0.922459,0.145287,3318.066162,3519.246826,3285.662698,3478.497947,32.403464,40.748879,17072.456996,17054.250063,1044.858928,731.538130
1,6558,EL CAJON,15381,15382,Primary,1.0,BA,0.123089,0.812051,1827.766479,2306.507080,1811.078249,2277.974003,16.688230,28.533077,13122.210664,13543.628848,447.335914,865.382417,0.123204,0.755244,3357.654541,2556.724854,3331.692010,2520.939386,25.962531,35.785468,16956.480996,16726.800133,1159.454136,828.060786
2,14726,TEXAS,15278,15381,Secondary,1.0,AB,0.973746,0.694543,1455.914551,1420.895630,1429.329664,1405.324334,26.584887,15.571296,8652.664006,4075.058951,60.890566,153.521820,1.173035,0.685165,2515.334961,1670.845215,2496.594145,1647.428578,18.740816,23.416637,11163.984239,5029.318481,243.473953,160.401404
3,9982,TEXAS,15381,15483,Secondary,1.0,BA,0.574607,0.819298,790.727234,988.457764,775.724325,976.616053,15.002909,11.841711,3456.867085,3591.401211,0.000000,0.000000,0.590455,0.826916,1083.600342,1241.220337,1074.268763,1228.690465,9.331579,12.529872,4442.295850,4439.350979,0.000000,0.000000
4,6559,EL CAJON,30914,15385,Primary,2.0,AB,0.502647,0.121749,2144.520264,3274.759766,2128.074322,3223.832262,16.445942,50.927504,13221.130576,13664.931195,461.835914,838.882417,0.546172,0.121778,3238.226074,4158.672852,3203.274465,4104.712459,34.951609,53.960393,16918.366347,16983.463821,1135.763432,840.370051
5,3659,EL CAJON,15385,15386,Primary,2.0,BA,0.124140,0.563079,2445.473877,2717.082275,2424.785880,2671.479871,20.687997,45.602404,13267.544906,13580.726025,461.835914,838.882417,0.124188,0.562496,3407.487305,3374.456787,3369.739675,3330.462647,37.747630,43.994140,16940.283996,16895.599863,1135.763432,840.370051
6,14716,UTAH,15280,15385,Secondary,2.0,AB,1.042639,0.556496,1128.394775,409.917847,1113.201485,405.346637,15.193290,4.571210,4022.568321,4150.604459,0.000000,0.000000,1.131059,0.563756,1630.974609,587.361145,1612.388001,582.145014,18.586608,5.216131,5125.965601,5121.070186,0.000000,0.000000
7,14735,UTAH,15385,15488,Secondary,2.0,BA,0.402519,0.732933,678.266174,818.420349,670.394721,811.603821,7.871453,6.816528,4024.339324,4142.998481,0.000000,0.000000,0.418258,0.744079,1136.418091,1046.282715,1125.139205,1035.612037,11.278886,10.670678,5129.536756,5111.922389,0.000000,0.000000
8,3660,EL CAJON,15386,15387,Primary,3.0,AB,0.536902,0.123535,2445.473877,2717.082275,2424.785880,2671.479871,20.687997,45.602404,13267.544906,13580.726025,461.835914,838.882417,0.564253,0.123534,3407.487305,3374.456787,3369.739675,3330.462647,37.747630,43.994140,16940.283996,16895.599863,1135.763432,840.370051
9,5236,EL CAJON,15387,15389,Primary,3.0,BA,0.480143,0.740514,3707.487061,3172.351074,3659.556977,3125.268669,47.930084,47.082405,13464.145341,13760.045951,523.223168,629.831247,0.485783,0.743544,4863.750977,3968.783691,4811.377258,3924.116619,52.373719,44.667072,17178.872074,17099.676820,893.787491,838.923842


In [27]:
# calculate total travel time for the corridor by direction 
ab_am_time = round(corridor_links_df[corridor_links_df["corridor"] == "Primary"]["ab_am_time"].sum(), 2) + (ab_transit_stops * AVG_DWELL_TIME_MIN)
ba_am_time = round(corridor_links_df[corridor_links_df["corridor"] == "Primary"]["ba_am_time"].sum(), 2) + (ba_transit_stops * AVG_DWELL_TIME_MIN)
ab_pm_time = round(corridor_links_df[corridor_links_df["corridor"] == "Primary"]["ab_pm_time"].sum(), 2) + (ab_transit_stops * AVG_DWELL_TIME_MIN)
ba_pm_time = round(corridor_links_df[corridor_links_df["corridor"] == "Primary"]["ba_pm_time"].sum(), 2) + (ba_transit_stops * AVG_DWELL_TIME_MIN)

In [28]:
ab_am_time, ba_am_time, ab_pm_time, ba_pm_time

(4.736666666666666, 4.58, 4.996666666666667, 4.51)

In [29]:
data_df = corridor_links_df.copy()
data_df = data_df.query("intersection > 0", engine="python")

data_df["am_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_flow"], data_df["ba_am_flow"])
data_df["pm_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_flow"], data_df["ba_pm_flow"])

data_df["am_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_auto_flow"], data_df["ba_am_auto_flow"])
data_df["pm_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_auto_flow"], data_df["ba_pm_auto_flow"])

data_df["am_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_truck_flow"], data_df["ba_am_truck_flow"])
data_df["pm_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_truck_flow"], data_df["ba_pm_truck_flow"])

data_df["am_capacity"] = np.where(data_df["approach"] == "AB", data_df["ab_am_capacity"], data_df["ba_am_capacity"])
data_df["pm_capacity"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_capacity"], data_df["ba_pm_capacity"])

data_df["am_transit_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_transit_flow"], data_df["ba_am_transit_flow"])
data_df["pm_transit_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_transit_flow"], data_df["ba_pm_transit_flow"])


In [30]:
voc_df = data_df.groupby(["intersection", "corridor"], as_index=False).agg({"am_flow": 'sum', "pm_flow": 'sum', "am_capacity": 'sum', "pm_capacity": 'sum'})
voc_df["am_voc"] = voc_df["am_flow"]/voc_df["am_capacity"]
voc_df["pm_voc"] = voc_df["pm_flow"]/voc_df["pm_capacity"]
voc_df = voc_df[["intersection", "corridor", "am_voc", "pm_voc"]]

In [31]:
data_df = pd.merge(data_df, voc_df, how="left", on=["intersection", "corridor"])

In [32]:
data_df = data_df[["hwycov_id", "from_node", "to_node", "link_name",
                   "corridor", "intersection", "approach", "am_auto_flow", "pm_auto_flow",
                   "am_truck_flow", "pm_truck_flow", "am_voc", "pm_voc", "am_transit_flow", "pm_transit_flow"]]

In [33]:
data_df

,hwycov_id,from_node,to_node,link_name,corridor,intersection,approach,am_auto_flow,pm_auto_flow,am_truck_flow,pm_truck_flow,am_voc,pm_voc,am_transit_flow,pm_transit_flow
0,6557,15380,15381,EL CAJON,Primary,1.0,AB,1958.778651,3285.662698,20.628209,32.403464,0.159616,0.173814,398.961856,1044.858928
1,6558,15381,15382,EL CAJON,Primary,1.0,BA,2277.974003,2520.939386,28.533077,35.785468,0.159616,0.173814,865.382417,828.060786
2,14726,15278,15381,TEXAS,Secondary,1.0,AB,1429.329664,2496.594145,26.584887,18.740816,0.199637,0.240753,60.890566,243.473953
3,9982,15381,15483,TEXAS,Secondary,1.0,BA,976.616053,1228.690465,11.841711,12.529872,0.199637,0.240753,0.000000,0.000000
4,6559,30914,15385,EL CAJON,Primary,2.0,AB,2128.074322,3203.274465,16.445942,34.951609,0.181391,0.195561,461.835914,1135.763432
5,3659,15385,15386,EL CAJON,Primary,2.0,BA,2671.479871,3330.462647,45.602404,43.994140,0.181391,0.195561,838.882417,840.370051
6,14716,15280,15385,UTAH,Secondary,2.0,AB,1113.201485,1612.388001,15.193290,18.586608,0.238418,0.261505,0.000000,0.000000
7,14735,15385,15488,UTAH,Secondary,2.0,BA,811.603821,1035.612037,6.816528,10.670678,0.238418,0.261505,0.000000,0.000000
8,3660,15386,15387,EL CAJON,Primary,3.0,AB,2424.785880,3369.739675,20.687997,37.747630,0.207855,0.216694,461.835914,1135.763432
9,5236,15387,15389,EL CAJON,Primary,3.0,BA,3125.268669,3924.116619,47.082405,44.667072,0.207855,0.216694,629.831247,838.923842


In [34]:
data_intersection_df = data_df.groupby(["intersection", "corridor"], as_index=False).agg({
    "am_auto_flow": 'sum',
    "am_truck_flow": 'sum',
    "am_transit_flow": 'sum',    
    "am_voc": 'mean',
    "pm_auto_flow": 'sum',
    "pm_truck_flow": 'sum',
    "pm_transit_flow": 'sum', 
    "pm_voc": 'mean'
})

In [35]:
data_intersection_df

,intersection,corridor,am_auto_flow,am_truck_flow,am_transit_flow,am_voc,pm_auto_flow,pm_truck_flow,pm_transit_flow,pm_voc
0,1.0,Primary,4236.752654,49.161286,1264.344273,0.159616,5806.602084,68.188932,1872.919714,0.173814
1,1.0,Secondary,2405.945717,38.426598,60.890566,0.199637,3725.284610,31.270688,243.473953,0.240753
2,2.0,Primary,4799.554193,62.048346,1300.718331,0.181391,6533.737112,78.945749,1976.133483,0.195561
3,2.0,Secondary,1924.805306,22.009818,0.000000,0.238418,2648.000038,29.257286,0.000000,0.261505
4,3.0,Primary,5550.054549,67.770402,1091.667161,0.207855,7293.856294,82.414702,1974.687274,0.216694
5,3.0,Secondary,1793.792071,38.504621,171.218083,0.235722,2382.572938,21.366882,209.455242,0.243821


In [36]:
data_avg_intersection_df = data_intersection_df.groupby(["corridor"], as_index=False).agg({
    "am_auto_flow": 'mean',
    "am_truck_flow": 'mean',
    "am_transit_flow": 'mean', 
    "am_voc": 'mean',
    "pm_auto_flow": 'mean',
    "pm_truck_flow": 'mean',
    "pm_transit_flow": 'mean', 
    "pm_voc": 'mean'
})

In [37]:
data_avg_intersection_df

,corridor,am_auto_flow,am_truck_flow,am_transit_flow,am_voc,pm_auto_flow,pm_truck_flow,pm_transit_flow,pm_voc
0,Primary,4862.120465,59.660011,1218.909922,0.182954,6544.731830,76.516461,1941.246824,0.195356
1,Secondary,2041.514365,32.980346,77.369550,0.224592,2918.619195,27.298285,150.976398,0.248693


In [38]:
data_avg_intersection_df.set_index('corridor', inplace=True)

work_df = pd.DataFrame.from_dict({
    "type": ["auto", "truck", "bus"],
    "am_flow_primary": [data_avg_intersection_df.loc["Primary"].at["am_auto_flow"], data_avg_intersection_df.loc["Primary"].at["am_truck_flow"], data_avg_intersection_df.loc["Primary"].at["am_transit_flow"]],
    "am_flow_secondary": [data_avg_intersection_df.loc["Secondary"].at["am_auto_flow"], data_avg_intersection_df.loc["Secondary"].at["am_truck_flow"], 0],
    "am_voc_primary": [data_avg_intersection_df.loc["Primary"].at["am_voc"], data_avg_intersection_df.loc["Primary"].at["am_voc"], data_avg_intersection_df.loc["Primary"].at["am_voc"]],
    "am_voc_secondary": [data_avg_intersection_df.loc["Secondary"].at["am_voc"], data_avg_intersection_df.loc["Secondary"].at["am_voc"], data_avg_intersection_df.loc["Secondary"].at["am_voc"]],
    "pm_flow_primary": [data_avg_intersection_df.loc["Primary"].at["pm_auto_flow"], data_avg_intersection_df.loc["Primary"].at["pm_truck_flow"], data_avg_intersection_df.loc["Primary"].at["pm_transit_flow"]],
    "pm_flow_secondary": [data_avg_intersection_df.loc["Secondary"].at["pm_auto_flow"], data_avg_intersection_df.loc["Secondary"].at["pm_truck_flow"], 0],
    "pm_voc_primary": [data_avg_intersection_df.loc["Primary"].at["pm_voc"], data_avg_intersection_df.loc["Primary"].at["pm_voc"], data_avg_intersection_df.loc["Primary"].at["pm_voc"]],
    "pm_voc_secondary": [data_avg_intersection_df.loc["Secondary"].at["pm_voc"], data_avg_intersection_df.loc["Secondary"].at["pm_voc"], data_avg_intersection_df.loc["Secondary"].at["pm_voc"]]
})

work_df["am_voc_primary"] = work_df["am_voc_primary"].apply(lambda x: min(1, x))
work_df["am_voc_secondary"] = work_df["am_voc_secondary"].apply(lambda x: min(1, x))
work_df["pm_voc_primary"] = work_df["pm_voc_primary"].apply(lambda x: min(1, x))
work_df["pm_voc_secondary"] = work_df["pm_voc_secondary"].apply(lambda x: min(1, x))

In [39]:
work_df

,type,am_flow_primary,am_flow_secondary,am_voc_primary,am_voc_secondary,pm_flow_primary,pm_flow_secondary,pm_voc_primary,pm_voc_secondary
0,auto,4862.120465,2041.514365,0.182954,0.224592,6544.731830,2918.619195,0.195356,0.248693
1,truck,59.660011,32.980346,0.182954,0.224592,76.516461,27.298285,0.195356,0.248693
2,bus,1218.909922,0.000000,0.182954,0.224592,1941.246824,0.000000,0.195356,0.248693


In [40]:
am_prob_bus_in_cycle = AVG_CYCLE_LENGTH / (60 * am_average_transit_headway)
pm_prob_bus_in_cycle = AVG_CYCLE_LENGTH / (60 * pm_average_transit_headway)

In [41]:
work_df["am_primary_delay_without_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_WITHOUT_TSP)**2)/ (1 - work_df["am_voc_primary"] * GREEN_TO_CYCLE_WITHOUT_TSP)
work_df["am_secondary_delay_without_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_WITHOUT_TSP)**2)/ (1 - work_df["am_voc_secondary"] * GREEN_TO_CYCLE_WITHOUT_TSP)
work_df["am_primary_delay_with_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_PRIMARY_WITH_TSP)**2)/ (1 - work_df["am_voc_primary"] * GREEN_TO_CYCLE_PRIMARY_WITH_TSP)
work_df["am_secondary_delay_with_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_SECONDARY_WITH_TSP)**2)/ (1 - work_df["am_voc_secondary"] * GREEN_TO_CYCLE_SECONDARY_WITH_TSP)

work_df["pm_primary_delay_without_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_WITHOUT_TSP)**2)/ (1 - work_df["pm_voc_primary"] * GREEN_TO_CYCLE_WITHOUT_TSP)
work_df["pm_secondary_delay_without_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_WITHOUT_TSP)**2)/ (1 - work_df["pm_voc_secondary"] * GREEN_TO_CYCLE_WITHOUT_TSP)
work_df["pm_primary_delay_with_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_PRIMARY_WITH_TSP)**2)/ (1 - work_df["pm_voc_primary"] * GREEN_TO_CYCLE_PRIMARY_WITH_TSP)
work_df["pm_secondary_delay_with_tsp"] = 0.5 * (AVG_CYCLE_LENGTH / 60) * ((1 - GREEN_TO_CYCLE_SECONDARY_WITH_TSP)**2)/ (1 - work_df["pm_voc_secondary"] * GREEN_TO_CYCLE_SECONDARY_WITH_TSP)

In [42]:
work_df["am_delay_savings"] = (
    (work_df["am_flow_primary"] * (work_df["am_primary_delay_without_tsp"] - work_df["am_primary_delay_with_tsp"]) * am_prob_bus_in_cycle) + 
    (work_df["am_flow_secondary"] * (work_df["am_secondary_delay_without_tsp"] - work_df["am_primary_delay_with_tsp"]) * am_prob_bus_in_cycle)
) / 3600

work_df["pm_delay_savings"] = (
    (work_df["pm_flow_primary"] * (work_df["pm_primary_delay_without_tsp"] - work_df["pm_primary_delay_with_tsp"]) * pm_prob_bus_in_cycle) + 
    (work_df["pm_flow_secondary"] * (work_df["pm_secondary_delay_without_tsp"] - work_df["pm_primary_delay_with_tsp"]) * pm_prob_bus_in_cycle)
) / 3600


In [43]:
work_df["total_delay_savings"] = (work_df["am_delay_savings"] * periods_length["AM"] + work_df["pm_delay_savings"] * periods_length["PM"]) * number_of_tsp_intersections

In [44]:
work_df["Variable"] = ["Delay Savings (hours) for Autos due to TSP", "Delay Savings (hours) for Trucks due to TSP", "Delay Savings (hours) for Buses due to TSP"]
work_df["Value"] = work_df["total_delay_savings"]

In [45]:
work_df

,type,am_flow_primary,am_flow_secondary,am_voc_primary,am_voc_secondary,pm_flow_primary,pm_flow_secondary,pm_voc_primary,pm_voc_secondary,am_primary_delay_without_tsp,am_secondary_delay_without_tsp,am_primary_delay_with_tsp,am_secondary_delay_with_tsp,pm_primary_delay_without_tsp,pm_secondary_delay_without_tsp,pm_primary_delay_with_tsp,pm_secondary_delay_with_tsp,am_delay_savings,pm_delay_savings,total_delay_savings,Variable,Value
0,auto,4862.120465,2041.514365,0.182954,0.224592,6544.731830,2918.619195,0.195356,0.248693,0.22931,0.234688,0.149774,0.329611,0.230886,0.237918,0.151037,0.33314,0.108945,0.150981,2.792270,Delay Savings (hours) for Autos due to TSP,2.792270
1,truck,59.660011,32.980346,0.182954,0.224592,76.516461,27.298285,0.195356,0.248693,0.22931,0.234688,0.149774,0.329611,0.230886,0.237918,0.151037,0.33314,0.001468,0.001650,0.033008,Delay Savings (hours) for Trucks due to TSP,0.033008
2,bus,1218.909922,0.000000,0.182954,0.224592,1941.246824,0.000000,0.195356,0.248693,0.22931,0.234688,0.149774,0.329611,0.230886,0.237918,0.151037,0.33314,0.018858,0.030152,0.531551,Delay Savings (hours) for Buses due to TSP,0.531551


In [46]:
delay_savings_df = work_df[["type", "total_delay_savings", "Variable", "Value"]]

In [47]:
delay_savings_df

,type,total_delay_savings,Variable,Value
0,auto,2.792270,Delay Savings (hours) for Autos due to TSP,2.792270
1,truck,0.033008,Delay Savings (hours) for Trucks due to TSP,0.033008
2,bus,0.531551,Delay Savings (hours) for Buses due to TSP,0.531551


In [48]:
# calculate reduction in VMT due to marginal mode shift to transit because of travel time reduction
am_primary_delay_without_tsp = work_df.loc[0].at["am_primary_delay_without_tsp"]
am_primary_delay_with_tsp = work_df.loc[0].at["am_primary_delay_with_tsp"]
am_pct_change_bus_travel_time = (am_primary_delay_with_tsp - am_primary_delay_without_tsp) * number_of_tsp_intersections /ab_am_time

pm_primary_delay_without_tsp = work_df.loc[0].at["pm_primary_delay_without_tsp"]
pm_primary_delay_with_tsp = work_df.loc[0].at["pm_primary_delay_with_tsp"]
pm_pct_change_bus_travel_time = (pm_primary_delay_with_tsp - pm_primary_delay_without_tsp) * number_of_tsp_intersections /ab_pm_time

In [49]:
am_new_transit_riders = am_pct_change_bus_travel_time * TT_ELASTICITY_TRANSIT_RIDERSHIP * am_avg_transit_flow * periods_length["AM"]
pm_new_transit_riders = pm_pct_change_bus_travel_time * TT_ELASTICITY_TRANSIT_RIDERSHIP * pm_avg_transit_flow * periods_length["PM"]

am_vmt_savings = am_avg_transit_trip_length * (am_new_transit_riders / AVG_AUTO_OCCUPANCY)
pm_vmt_savings = pm_avg_transit_trip_length * (pm_new_transit_riders / AVG_AUTO_OCCUPANCY)

total_vmt_savings = am_vmt_savings + pm_vmt_savings

In [50]:
am_vmt_savings, pm_vmt_savings, total_vmt_savings

(176.83714113008935, 326.3515915545729, 503.1887326846622)

In [51]:
vmt_savings_df = pd.DataFrame.from_dict(
    {
        'Variable': ["VMT AM savings due to mode shift to transit", "VMT PM savings due to mode shift to transit", "VMT TOTAL savings due to mode shift to transit"],
        'Value': [am_vmt_savings, pm_vmt_savings, total_vmt_savings]
    }
)

In [52]:
vmt_savings_df

,Variable,Value
0,VMT AM savings due to mode shift to transit,176.837141
1,VMT PM savings due to mode shift to transit,326.351592
2,VMT TOTAL savings due to mode shift to transit,503.188733


In [53]:
# get emission factors
emission_factors_df = get_emission_factors(emission_df, scen_year)

In [54]:
emission_factors_df

,Year,Vehicle Type,CO2 RunEx Emission Factor (gr/mile),CO2 IdlEx Emission Factor (gr/hour),CO2 StrEx Emission Factor (gr/trip)
0,2035,Passenger Car,213.69,518.91,43.24
1,2035,Bus - All Fuel Types,NaN,2622.82,NaN
2,2035,Bus - Dsl,0.00,0.00,0.00
3,2035,Bus - Gas,1254.85,0.00,45.34
4,2035,Bus - NG,2053.53,0.00,0.00
5,2035,Bus - Propane,NaN,NaN,NaN
6,2035,HDT - All Weight Types,NaN,2016.87,NaN
7,2035,Light HDT,644.42,121.78,7.83
8,2035,Medium HDT,937.84,570.07,8.13
9,2035,Heavy HDT,1140.56,11763.95,0.07


In [55]:
# calculate ghg reductions 
ghg_reduction_df = calculate_ghg_reduction(delay_savings_df, total_vmt_savings, emission_factors_df)

In [56]:
ghg_reduction_df

,Variable,Value
0,GHG reduction due to delay savings (short tons),0.003201
1,GHG reduction due to vmt savings (short tons),0.118279
2,Total GHG reduction (short tons),0.121480


In [57]:
delay_savings_df = delay_savings_df[["Variable", "Value"]]

In [58]:
# writing results
results_dict = {"GHG_Reduction": ghg_reduction_df, "VMT_Reduction": vmt_savings_df, "Delay_Reduction": delay_savings_df, "Emission_Factors": emission_factors_df}
write_results(results_dict, output_results_filename, output_dir)